In [94]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from collections import Counter
import collections 

In [95]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
262,tt0844471,100000000,242988466,Cloudy with a Chance of Meatballs,Bill Hader|Anna Faris|James Caan|Mr. T|Benjami...,Phil Lord|Christopher Miller,Prepare to get served.,Inventor Flint Lockwood creates a machine that...,90,Animation|Comedy|Family,Columbia Pictures|Sony Pictures Animation,9/17/2009,6.4,2009
139,tt2103281,170000000,708200000,Dawn of the Planet of the Apes,Andy Serkis|Jason Clarke|Toby Kebbell|Keri Rus...,Matt Reeves,One last chance for peace.,A group of scientists in San Francisco struggl...,130,Science Fiction|Action|Drama|Thriller,Ingenious Media|Chernin Entertainment|TSG Ente...,6/26/2014,7.4,2014
1698,tt0481369,30000000,77566815,The Number 23,Jim Carrey|Virginia Madsen|Logan Lerman|Danny ...,Joel Schumacher,First it takes hold of your mind...then it tak...,Walter Sparrow is an animal control officer th...,101,Thriller|Drama|Mystery,New Line Cinema|Contrafilm|Firm Films,2/23/2007,6.2,2007
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,8.0,2015
1443,tt0457939,85000000,194168700,The Holiday,Cameron Diaz|Kate Winslet|Jude Law|Jack Black|...,Nancy Meyers,It's Christmas Eve and we are going to go cele...,"Two women, one (Cameron Diaz) from America and...",136,Comedy|Romance,Columbia Pictures|Universal Pictures|Waverly F...,12/8/2006,6.7,2006


In [96]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [97]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

In [98]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


### Check if there any non null values

In [99]:
data.isnull().values.any()

False

### Check if columns of type 'object' are of the right data format

In [100]:
object_data = data.select_dtypes(include='object').columns
object_data

Index(['imdb_id', 'original_title', 'cast', 'director', 'tagline', 'overview',
       'genres', 'production_companies', 'release_date'],
      dtype='object')

In [101]:
for ob in object_data:
    print(type(ob),'\t', ob)

<class 'str'> 	 imdb_id
<class 'str'> 	 original_title
<class 'str'> 	 cast
<class 'str'> 	 director
<class 'str'> 	 tagline
<class 'str'> 	 overview
<class 'str'> 	 genres
<class 'str'> 	 production_companies
<class 'str'> 	 release_date


### Change columns with the wrong data format to the right one

In [102]:
data['release_date'] = pd.to_datetime(data['release_date'])
type(data['release_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

### Create a function collecting all answers to questions

In [103]:
answers = {}
def answer(answer, question = 1, name = True):
    if name:
        answers[question] = answer['original_title'].item() + ' (' + answer['imdb_id'].item() + ')'
    else: 
        answers[question] = answer
answers

{}

### Add profit data to the dataframe

In [104]:
data['profit'] = data['revenue'] - data['budget']
data['profit']

0       1363528810
1        228436354
2        185238201
3       1868178225
4       1316249360
           ...    
1884      82299717
1885      -8444012
1886     -40865180
1887      -3962091
1888      -9782502
Name: profit, Length: 1889, dtype: int64

### Function to convert lists to pd.Series 

In [105]:
def list_to_Series(x): 
    idx, values = zip(*x)
    return pd.Series(values, idx)

### Format the month df to make it work easier with it

In [264]:
def format_month_df(months):
    months.rename(columns = {'release_date':'count'}, inplace = True)
    months.index.names = ['release_month']
    month_names = ['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'Ocotber', 'November', 'December']
    months.index = map(lambda x: month_names[x-1], months.index)
    
    return months

# 1. У какого фильма из списка самый большой бюджет?

In [106]:
max_budget = data[data['budget'] == data['budget'].max()]
answer(max_budget)

answers[1]

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [107]:
longest = data[data['runtime'] == data['runtime'].max()]
answer(longest, 2)

answers[2]

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [108]:
shortest = data[data['runtime'] == data['runtime'].min()]
answer(shortest, 3)

answers[3]

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [109]:
avg_runtime = data['runtime'].mean()
answer(avg_runtime, 4, False)

answers[4]

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [110]:
meadian_runtime = data['runtime'].median()
answer(meadian_runtime, 5, False)

answers[5]

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [111]:
most_profit = data[data['profit']==data['profit'].max()]
answer(most_profit, 6)

answers[6]

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [112]:
least_profit = data[data['profit']==data['profit'].min()]
answer(least_profit, 7)

answers[7]

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [113]:
pos_profit = data[data['profit']>0].count()['imdb_id']
answer(pos_profit, 8, False)

answers[8]

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [114]:
movies_2008 = data[(data['release_year']==2008)]
highest_gross =  movies_2008[movies_2008['revenue']==movies_2008['revenue'].max()]
answer(highest_gross, 9)

answers[9]

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [115]:
period = data[(data['release_year']>=2012) | (data['release_year']<=2012)]
most_loss = period[period['revenue']==period['revenue'].min()]
answer(most_loss, 10)

answers[10]

'Mutant Chronicles (tt0490181)'

# 11. Какого жанра фильмов больше всего?

In [116]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
genres = data.groupby(['genres']).size().sort_values(ascending=False)
most_popular = genres[0]
print(genres)

answer(genres.index[0], 11, False)

answers[11]

genres
Comedy                            141
Drama                             102
Comedy|Romance                     60
Drama|Romance                      45
Comedy|Drama|Romance               35
                                 ... 
Drama|Mystery|Thriller|Fantasy      1
Drama|Mystery|Thriller|Horror       1
Drama|Romance|Comedy                1
Drama|Romance|Comedy|Music          1
Crime|Comedy|Romance                1
Length: 652, dtype: int64


'Comedy'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [117]:
profitable = data[data.profit > 0]
quantity = collections.Counter(profitable['genres'].str.cat(sep = '|').split('|')).most_common()

display(quantity)

answer(quantity[0][0], 12, False)

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337),
 ('Romance', 242),
 ('Crime', 231),
 ('Family', 226),
 ('Science Fiction', 195),
 ('Fantasy', 188),
 ('Horror', 150),
 ('Animation', 120),
 ('Mystery', 119),
 ('Music', 47),
 ('History', 46),
 ('War', 41),
 ('Western', 12),
 ('Documentary', 7)]

In [118]:
# profitable = data[data['profit']>0]
# profitable = profitable.groupby(['genres'])['profit'].sum().sort_values(ascending=False)
# profitable

In [119]:
# genres = list(set(profitable.index.str.cat(sep = '|').split('|')))
# init_values = [0] * len(genres)
# #  using series instead of a dictionary because series can be easily sorted
# popular_genres = pd.Series(init_values, index=genres) 
# popular_genres

In [120]:
# for name, profit in zip(profitable.index, profitable.values):
#     for n in name.split('|'):
#         popular_genres[n] += profit
# print(popular_genres.sort_values(ascending=False).head(5))

In [121]:
# for n, t in zip(popular_genres.index, popular_genres.values):
#     popular_genres[n] = t / quantity[n]
    
    
# popular_genres.sort_values(ascending=False)

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [122]:
directors = data.groupby(['director'])
movie_shark = directors['revenue'].sum().sort_values(ascending = False)
answer(movie_shark.index[0], 13, False)

movie_shark.head(5)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [123]:
action = data[data['genres'].str.contains('Action')][['director','genres']]
names = action['director'].str.cat(sep = '|').split('|')
directs_most = list_to_Series(collections.Counter(names).most_common())

display(directs_most.head())

answer(directs_most.index[0],14, False)

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Antoine Fuqua         6
Ridley Scott          6
dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [124]:
movies_12 = data[data['release_year']==2012][['original_title', 'cast', 'revenue']]
mean_revenue = movies_12['revenue'].mean()

top_movies = movies_12[movies_12['revenue']>mean_revenue].sort_values(['revenue'],ascending=False)

actors = top_movies['cast'].str.cat(sep = '|').split(sep='|')
pop_actors = list_to_Series(collections.Counter(actors).most_common())

display(pop_actors.head())

answer(pop_actors.index[0], 15, False)

Chris Hemsworth        2
Ralph Fiennes          2
Anne Hathaway          2
Denis Leary            2
Seann William Scott    2
dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [125]:
high_budget = data[data['budget'] > data['budget'].mean()][['original_title','cast','budget']]
actors = high_budget['cast'].str.cat(sep='|').split(sep='|')

most_pop_actor = list_to_Series(collections.Counter(actors).most_common())
most_pop_actor.head(3)

answer(most_pop_actor.index[0], 16, False)

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [126]:
cages_movies = data[data['cast'].str.find('Nicolas Cage')>=0][['original_title', 'cast', 'genres']]
genres = cages_movies['genres'].str.cat(sep='|').split(sep='|')

cages_pop_genre = list_to_Series(collections.Counter(genres).most_common())
cages_pop_genre.head()

answer(cages_pop_genre.index[0], 17, False)

# 18. Самый убыточный фильм от Paramount Pictures

In [127]:
paramount = data[data['production_companies'].str.find('Paramount Pictures')>=0][['imdb_id','original_title','profit']]
unsuccessful = paramount[paramount['profit']==paramount['profit'].min()]

answer(unsuccessful, 18)

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [128]:
revenue = data.groupby('release_year')['revenue'].sum().sort_values(ascending=False)
display(revenue)

answer(revenue.index[0], 19, False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [129]:
WB_movies = data[data['production_companies'].str.find('Warner Bros')>0][['release_year','profit']]
most_profit_year = WB_movies.groupby('release_year').sum().sort_values(['profit'],ascending=False)

display(most_profit_year.head(3))

answer(most_profit_year.index[0], 20, False)

,profit
release_year,
2014,2297979392
2008,2134595031
2007,1928180689


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [311]:
months = data['release_date'].apply(lambda x: x.month)
months = format_month_df(months.groupby(months).count().to_frame())

highest_num = months.sort_values(by = ['count'], ascending = False)
# answer(months.index[0],21, False)
# answers[21]

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [292]:
values = months[5:8]['count'].values
summer_mov = sum(values)
summer_mov

450

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [130]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{1: 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 2: 'Gods and Generals (tt0279111)',
 3: 'Winnie the Pooh (tt1449283)',
 4: 109.6585494970884,
 5: 107.0,
 6: 'Avatar (tt0499549)',
 7: 'The Lone Ranger (tt1210819)',
 8: 1478,
 9: 'The Dark Knight (tt0468569)',
 10: 'Mutant Chronicles (tt0490181)',
 11: 'Comedy',
 12: 'Drama',
 13: 'Peter Jackson',
 14: 'Robert Rodriguez',
 15: 'Chris Hemsworth',
 16: 'Matt Damon',
 17: 'Action',
 18: 'K-19: The Widowmaker (tt0267626)',
 19: 2015,
 20: 2014}

In [131]:
# и убедиться что ни чего не пропустил)
len(answers)

20